In [1]:
import os
import numpy as np
import pydicom

import pandas as pd
import sys
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/read_weasis_raw_v0.96/')

import weasis_raw_data_api as wr
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/')
from utils_test import *
from utils_metrics_3d import *

D_dir2header_df = {}
def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:

                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        
                        if metadata in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                            print( 'error of loading key: {}'.format(metadata) )                    
                        else:
                            data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [2]:
def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        try:
            df[col] = df[col].str.replace(ori,new, case = False) 
        except:
            pass
            
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instance should be str or list')


def str_Xdrive2mnt(df_all):
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/Y-drive")
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")

    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen",
    "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    
def get_onect_from_list(df_list , ct):
    for i, df in enumerate(df_list):
        try:
            df_ct = df[ (df["Image File Path"]==ct) & (df['Location'].isin(['liver'])) ]
        except KeyError:
            df_ct = df[ (df["Image File Path"]==ct)]
        if df_ct.shape[0]:
            return df_ct , i
    print("warning! no CT was found")
    return None ,None

def raws2mask(raws , D_z_index, mask_vol = None):

    for raw in raws:

        radiologist_raw = wr.read(raw)
        slice_list = radiologist_raw.get_instance_number_array()
        if mask_vol is None:
            mask_vol = initialize_mask_vol(radiologist_raw , D_z_index)
        for j, one in enumerate(slice_list):
            mask = radiologist_raw.get_mask_image(j)
            mask_vol[D_z_index[one]] += mask
    return mask_vol

In [3]:
subsetname = 'CUIMClast'
folder = '/mnt/fast-data/mjc/AutoRECIST/Inputs/'

df_CTs = pd.read_excel(folder+'AutoRECIST_List_LesionSize_20220602_JM_SingleCTSeries.xlsx')
str_Xdrive2mnt(df_CTs)

df_CTs = df_CTs[df_CTs['dataset']==subsetname]
print(df_CTs)

                                          Comments  Patient ID  \
225                         minimal or no revision  METNET2910   
226                         minimal or no revision  METNET2911   
227                                 minor revision  METNET2914   
228                                 minor revision  METNET2915   
229                                 major revision  METNET2917   
..                                             ...         ...   
348  minor revision(falsely targeted fat deposits)  METNET6058   
349                         minimal or no revision  METNET6067   
350                         minimal or no revision  METNET6070   
351                         minimal or no revision  METNET6073   
352                         minimal or no revision  METNET6082   

                                       Image File Path  \
225  /mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...   
226  /mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...   
227  /mnt/fast-disk1/mjc/AutoReci

In [ ]:
df_Yen = pd.read_excel(folder+'PDS_AUTO_RECIST CIA-LAB Testing Dataset Gold Standard_Yen_2022-06-13.xlsx')
str_Xdrive2mnt(df_Yen)

AI_raw_list = ['ScaleNAS9SlicesAccLiverToRaw_Test353.csv',]
df_AIs = []
for one in AI_raw_list:
    df = pd.read_csv(one)
    str_Xdrive2mnt(df)
    df_AIs.append(df)




Metrics_vol = []
metrics_save_path = 'Metrics_%s_vs_Yen_%s.csv'%('ScaleNAS9SlicesAccLiver' ,subsetname )
CTs = df_CTs["Image File Path"].values.tolist()

for ct in CTs:
    df_image = get_dicom_header_df( ct )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)


    df_ct_Yen = df_Yen[df_Yen["Image File Path"]==ct]
    df_ct_AI , dataset_id = get_onect_from_list(df_AIs , ct)
    # break
    if (df_ct_AI is None):
        if df_ct_Yen.shape[0]:
            fn = df_ct_Yen.shape[0]
            print('{} has {} FNs!'.format(ct , fn))   
        continue

    if not df_ct_Yen.shape[0]:
        fp = df_ct_AI.shape[0]
        print( '{} has {} FPs'.format(ct , fp)  )
        continue
    else:
        print(ct)

    raws = df_ct_AI["Contour File Path"].values.tolist()
    vol_pred = raws2mask(raws , D_z_index, mask_vol = None)
    connectivity = 2
    from skimage import measure
    labels_pred=measure.label(vol_pred,connectivity=connectivity)
    l_pred,c_pred = np.unique(labels_pred , return_counts=True)
    ix2 = l_pred>0
    l_pred = l_pred[ix2] #background pixels are labeled as 0, so we exclude them
    c_pred = c_pred[ix2]
    if len(l_pred)!= len(raws):
        print("warning! raws overlaped on {}".format(ct))


    for _ , row in df_ct_Yen.iterrows():

        Yen_raw = wr.read(row['Contour File Path'])
        gt_vol = initialize_mask_vol(Yen_raw , D_z_index)

        slice_list = Yen_raw.get_instance_number_array()
        for j, one in enumerate(slice_list):
            mask = Yen_raw.get_mask_image(j)
            gt_vol[D_z_index[one]] = mask
        
        hit = vols_seg_results(gt_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
        Metrics_vol.extend(hit)

        _n = len(Metrics_vol)
        if _n%100==0 or _n in [1,2,5,10,30,50]:
            df_metrics = pd.DataFrame(Metrics_vol, 
                                    columns = ['file_name','igt','merge','#gt','#pred',
                                                'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                                'area_gt','area_pred','intersection','union']) 
            df_metrics.to_csv(metrics_save_path)
        

/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2910/D2015_02_06/E8602/CT/S0005_3151
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2911/D2015_02_13/E2523/CT/S0004_1732
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2914/D2015_03_16/E8300/CT/S0002_0406
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2915/D2015_03_19/E4011/CT/S0002_6559
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2917/D2015_04_02/E6773/CT/S0009_6114
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2919/D2015_04_15/E0185/CT/S0003_1069
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2922/D2015_04_23/E6734/CT/S0003_4852
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2929/D2015_07_09/E4714/CT/S0004_9464
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2931/D2015_09_09/E2142/CT/S0011_8945
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET2934/D2015_04_24/E4630/CT/S0004_8438
/mnt/fast-disk1/mjc/AutoRecist

In [12]:
df_metrics = pd.DataFrame(Metrics_vol, 
                        columns = ['file_name','igt','merge','#gt','#pred',
                                    'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                    'area_gt','area_pred','intersection','union']) 
df_metrics.to_csv(metrics_save_path)

In [13]:
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000,1016.000
mean,1.011,0.000,1.000,11.223,0.293,0.374,3.928,0.600,11922.523,10166.925,7228.519,14860.929
std,0.113,0.000,0.000,7.779,0.294,0.350,52.963,0.380,62315.998,43949.089,38514.984,66054.053
min,1.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,9.000,0.000,0.000,9.000
5%,1.000,0.000,1.000,2.000,0.000,0.000,0.000,0.023,60.000,0.000,0.000,65.750
25%,1.000,0.000,1.000,5.000,0.000,0.000,0.000,0.215,198.750,0.000,0.000,233.000
50%,1.000,0.000,1.000,10.000,0.225,0.367,0.017,0.633,719.500,350.500,223.500,975.000
75%,1.000,0.000,1.000,16.000,0.570,0.726,0.226,1.000,3448.000,2704.750,1572.000,5120.000
95%,1.000,0.000,1.000,25.000,0.792,0.884,2.487,1.000,36274.500,42091.750,24639.750,62506.750
max,3.000,0.000,1.000,35.000,0.904,0.949,1505.350,1.000,862456.000,569916.000,521180.000,867354.000


In [14]:
df_metrics[df_metrics.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,555.000,555.000,555.000,555.000,555.000,555.000,555.000,555.000,555.000,555.000,555.000,555.000
mean,1.005,0.000,1.000,11.888,0.527,0.669,0.370,0.319,20222.946,15036.213,13122.802,22136.357
std,0.073,0.000,0.000,8.150,0.191,0.175,0.657,0.243,83111.749,54607.959,51388.615,85582.670
min,1.000,0.000,1.000,1.000,0.146,0.255,0.000,0.000,31.000,25.000,25.000,59.000
5%,1.000,0.000,1.000,2.000,0.198,0.331,0.000,0.012,126.000,113.000,73.800,202.700
25%,1.000,0.000,1.000,6.000,0.380,0.551,0.032,0.112,549.000,501.500,346.000,782.500
50%,1.000,0.000,1.000,10.000,0.549,0.708,0.129,0.262,1919.000,1687.000,1214.000,2478.000
75%,1.000,0.000,1.000,16.000,0.681,0.810,0.389,0.500,8077.500,6621.000,5215.000,10051.500
95%,1.000,0.000,1.000,28.000,0.813,0.897,1.593,0.775,75239.100,58386.100,47270.200,86739.100
max,2.000,0.000,1.000,35.000,0.904,0.949,5.467,0.854,862456.000,569916.000,521180.000,867354.000


In [15]:
dfinnermerge = pd.merge(df_metrics,df_Yen,how='inner',left_on='file_name' , right_on='Contour File Path')
for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

file_name 1016
igt 3
merge 1
#gt 1
#pred 21
iou_score 642
dice_score 642
over_seg 597
under_seg 625
area_gt 829
area_pred 555
intersection 578
union 854
Image File Path 127
Contour File Path 1016
Raw File Name 1016
Uni 922
Perp 889
Bi 993
Volume 1002
In total, 127 CT series; 796 AI detections 
sensitivity is 0.631(641/1016) FP-rate is 1.2 per CT-serie at threshold 0
sensitivity is 0.588(597/1016) FP-rate is 1.6 per CT-serie at threshold 0.1
sensitivity is 0.559(568/1016) FP-rate is 1.8 per CT-serie at threshold 0.2
sensitivity is 0.546(555/1016) FP-rate is 1.9 per CT-serie at threshold 0.25
sensitivity is 0.442(449/1016) FP-rate is 2.7 per CT-serie at threshold 0.5


In [16]:
#subgroup analysis for gt_lesion >=10mm

Uni_thresh = 10
df_subgroup = dfinnermerge[dfinnermerge['Uni']>=Uni_thresh]
df_subgroup.describe([.05, .25, .5, .75, .95])
df_subgroup[df_subgroup.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union,Uni,Perp,Bi,Volume
count,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000,481.000
mean,1.004,0.000,1.000,11.401,0.538,0.679,0.296,0.327,23298.628,17302.252,15116.734,25484.146,32.665,22.654,1170.286,36906.195
std,0.064,0.000,0.000,7.432,0.190,0.174,0.507,0.244,88889.683,58336.531,54936.576,91483.989,26.959,16.853,2509.464,166419.601
min,1.000,0.000,1.000,1.000,0.146,0.255,0.000,0.000,89.000,36.000,36.000,130.000,10.067,3.985,48.239,137.810
5%,1.000,0.000,1.000,2.000,0.202,0.336,0.001,0.017,324.000,222.000,156.000,410.000,11.389,7.229,82.288,402.965
25%,1.000,0.000,1.000,6.000,0.407,0.578,0.028,0.117,947.000,786.000,535.000,1226.000,15.564,11.855,180.419,1233.820
50%,1.000,0.000,1.000,10.000,0.568,0.724,0.111,0.286,2452.000,2249.000,1692.000,3102.000,23.116,17.827,405.287,3849.160
75%,1.000,0.000,1.000,16.000,0.687,0.815,0.347,0.513,10038.000,8480.000,6320.000,12635.000,39.564,28.690,1149.079,17338.100
95%,1.000,0.000,1.000,23.000,0.819,0.901,1.254,0.775,82010.000,69234.000,57124.000,97333.000,86.154,52.779,4367.996,112044.000
max,2.000,0.000,1.000,35.000,0.904,0.949,3.982,0.854,862456.000,569916.000,521180.000,867354.000,202.413,134.942,27314.014,2618860.000


In [17]:
def detection_performance(dfinnermerge):
    pts = dfinnermerge["Image File Path"].values.tolist()
    FPs = []
    for onept in list(set(pts)):
        df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
        assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
        fp = max(df_onept["#pred"])
        FPs.append(fp)
    print("="*80)
    print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

    dices = dfinnermerge.dice_score.tolist()

    for th in [0, 0.1, 0.2, 0.25, 0.5]:
        TP = [p>th for p in dices ]
        assert( len(TP) == len(dices))
        fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
        print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

In [18]:
detection_performance(df_subgroup)

In total, 123 CT series; 790 AI detections 
sensitivity is 0.741(546/737) FP-rate is 2.0 per CT-serie at threshold 0
sensitivity is 0.700(516/737) FP-rate is 2.2 per CT-serie at threshold 0.1
sensitivity is 0.665(490/737) FP-rate is 2.4 per CT-serie at threshold 0.2
sensitivity is 0.653(481/737) FP-rate is 2.5 per CT-serie at threshold 0.25
sensitivity is 0.535(394/737) FP-rate is 3.2 per CT-serie at threshold 0.5
